# NI USB-6002 DAQ (DAQmx → Python)

This notebook is focused on **Single Channel data acquisition** (DAQ): detecting an NI device, collecting a clean number of samples, optionally viewing data live, and exporting a CSV + metadata.

**Workflow:**
1. Overview
2. Hardware setup + wiring / channel map
3. Install + imports
4. DAQ Config
5. Acquire
6. Live view
7. Save data + metadata + Post plot

> Requires NI-DAQmx driver (device visible in NI MAX) and the Python package `nidaqmx`. Will only work on Windows.

## 2. Hardware setup + wiring / channel map

**Channel map**
- `Dev1/ai0` → `ch0_V`

**Wiring (typical)**
- Connect your signal to AI0
- Connect ground/reference appropriately based on terminal configuration (RSE vs Differential)

**Safety**
- Keep input within the configured voltage range (`min_v` to `max_v`).

## 3. Install + imports

Recommended conda install (after NI-DAQmx is installed):

```bash
conda install -c conda-forge nidaqmx ipympl pandas matplotlib numpy
```

Then restart the kernel.

In [1]:
# Interactive plotting backend (recommended for live view)
%matplotlib widget

from datetime import datetime

import numpy as np
import pandas as pd

import nidaqmx
from nidaqmx.constants import AcquisitionType, TerminalConfiguration
from nidaqmx.stream_readers import AnalogSingleChannelReader
from nidaqmx.system import System

import matplotlib.pyplot as plt


ModuleNotFoundError: No module named 'nidaqmx'

## 4. DAQ Config
# Detect NI DAQ devices
This should list your device (often `Dev1`). If nothing appears, check:
- USB connection
- NI-DAQmx installation
- NI MAX can see the device


In [2]:
system = System.local()
print('NI-DAQmx driver version:', system.driver_version)

if len(system.devices) == 0:
    raise RuntimeError('No NI DAQ devices found. Check NI MAX and your USB connection.')

for dev in system.devices:
    print('Device:', dev.name, '| Product:', dev.product_type)


NameError: name 'System' is not defined

# Acquisition settings
Edit these values as needed.

**Sampling**:
- `SAMPLE_RATE_HZ` The **rate** at which you'd like to sample
- `DURATION_S` How long to run samples for

**Input Range**:
- By Default this is set to -10.0 to 10.0 V. The range is divided by the ADC (2^16 bins) so the resolution is 0.31 mV. Note: On some devices you can change the input range for higher resoluiton, however on the USB6002 changing the input range will have no affect on the resolution.
  
**Terminal configuration** depends on wiring:
- `RSE` (Referenced Single-Ended) is common for quick single-ended measurements referenced to AI GND. This is analogous to `analogRead` on Arduino
- `DIFFERENTIAL` can reduce noise if wired differentially.


In [9]:
# --- Core DAQ config (shared names across notebooks) ---
sample_rate_hz = 1000.0       # NI: hardware-enforced
duration_s = 10.0             # seconds
n_samples = int(round(sample_rate_hz * duration_s))

chunk_samples = 200           # NI read chunk size (also UI update size for live view)
plot_window_s = 5.0           # seconds shown in the live view window
live_view = True

channels = ['ch0_V']
units = {'ch0_V': 'V'}

# --- NI-specific config ---
device_name = 'Dev1'
ai_channel = 'ai0'
phys_chan = f'{device_name}/{ai_channel}'

min_v = -10.0
max_v = 10.0
TERM_CFG = TerminalConfiguration.RSE
# TERM_CFG = TerminalConfiguration.DIFFERENTIAL

print(f'n_samples = {n_samples}')


NameError: name 'TerminalConfiguration' is not defined

## 5. Acquire
This section acquires **exactly** `n_samples` using the NI hardware sample clock, and stores results in memory.

Acquisition is decoupled from plotting. The live view (Section 6) only *visualizes* a rolling buffer.


In [ ]:
def acquire_ni_single_channel(n_samples: int, live_callback=None):
    """Acquire n_samples from a single NI analog input.

    Parameters
    ----------
    n_samples : int
        Total samples to acquire.
    live_callback : callable or None
        If provided, called as live_callback(t_chunk, v_chunk) after each chunk.

    Returns
    -------
    df : pandas.DataFrame with columns time_s, ch0_V
    """
    data = np.empty(n_samples, dtype=np.float64)

    with nidaqmx.Task() as task:
        task.ai_channels.add_ai_voltage_chan(
            phys_chan,
            min_val=min_v,
            max_val=max_v,
            terminal_config=terminal_config,
        )

        task.timing.cfg_samp_clk_timing(
            rate=sample_rate_hz,
            sample_mode=AcquisitionType.FINITE,
            samps_per_chan=n_samples,
        )

        reader = AnalogSingleChannelReader(task.in_stream)

        # Start + read in chunks
        task.start()
        idx = 0
        while idx < n_samples:
            n = min(chunk_samples, n_samples - idx)
            buf = np.empty(n, dtype=np.float64)
            reader.read_many_sample(buf, number_of_samples_per_channel=n, timeout=10.0)
            data[idx:idx+n] = buf

            if live_callback is not None:
                t0 = idx / sample_rate_hz
                t_chunk = t0 + np.arange(n) / sample_rate_hz
                live_callback(t_chunk, buf)

            idx += n

    t = np.arange(n_samples, dtype=np.float64) / sample_rate_hz
    df = pd.DataFrame({'time_s': t, 'ch0_V': data})
    return df


df = acquire_ni_single_channel(n_samples)
print(df.head())
print(f'Collected {len(df)} samples')


## 6. Live view
If you want live plotting while sampling, use the cell below.

**Tip:** Live view is best for checking wiring and signal sanity. Your saved dataset will still include all samples.



In [ ]:
# Live view acquisition (optional)
# This re-acquires data. If you already ran Section 5, you can skip this section.

if not live_view:
    print('live_view is False — skipping live view acquisition.')
else:
    maxlen = int(max(10, round(plot_window_s * sample_rate_hz)))
    tbuf = deque(maxlen=maxlen)
    vbuf = deque(maxlen=maxlen)

    fig, ax = plt.subplots()
    (line,) = ax.plot([], [])
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Voltage (V)')
    ax.grid(True)

    def _update_live(t_chunk, v_chunk):
        for tt, vv in zip(t_chunk, v_chunk):
            tbuf.append(tt)
            vbuf.append(vv)
        line.set_data(list(tbuf), list(vbuf))
        ax.relim()
        ax.autoscale_view()
        fig.canvas.draw_idle()
        plt.pause(0.001)

    from collections import deque
    df = acquire_ni_single_channel(n_samples, live_callback=_update_live)
    print(f'Collected {len(df)} samples with live view')


## 7. Save data + metadata + post plot
This section exports a CSV (Excel-friendly) and a small JSON metadata file.

In [ ]:
run_id = datetime.now().strftime('%Y%m%d_%H%M%S')
base = f'ni_usb6002_daq_{run_id}'
csv_path = base + '.csv'
json_path = base + '.json'

# Save CSV
df[['time_s', 'ch0_V']].to_csv(csv_path, index=False)
print('Saved CSV:', csv_path)

# Save metadata JSON
metadata = {
    'device_type': 'ni_usb_6002',
    'requested_sample_rate_hz': sample_rate_hz,
    'effective_sample_rate_hz': sample_rate_hz,
    'duration_s': duration_s,
    'n_samples': int(len(df)),
    'channels': channels,
    'units': units,
    'phys_chan': phys_chan,
    'min_v': min_v,
    'max_v': max_v,
    'terminal_config': str(terminal_config),
}

import json
with open(json_path, 'w') as f:
    json.dump(metadata, f, indent=2)
print('Saved metadata:', json_path)

# Post plot (static)
plt.figure()
plt.plot(df['time_s'], df['ch0_V'])
plt.xlabel('Time (s)')
plt.ylabel('Voltage (V)')
plt.title(f'NI USB-6002 | {len(df)} samples | {sample_rate_hz} Hz')
plt.grid(True)
plt.show()
